In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
users = pd.read_csv('data/USERS.csv')
qs = pd.read_csv('data/QUESTIONNAIRE.csv')
motion = pd.read_csv('data/MOTION.csv')
sound = pd.read_csv('data/SOUND.csv')
location = pd.read_csv('data/LOCATION.csv')

In [4]:
users = users[users['userID'] >= 30]
qs = qs[qs['user'] >= 30]
motion = motion[motion['user'] >= 30]
sound = sound[sound['user']>= 30]
location = location[location['user'] >= 30]

In [5]:
users = users.rename(columns={'userID':'user'})
joined = pd.merge(qs,users,on='user',how='left')

In [6]:
motion.columns

Index(['user', 'datetime', 'motionEntryID', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6',
       'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16',
       'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26',
       'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35'],
      dtype='object')

In [7]:
motion.isna().sum()

user                  0
datetime              0
motionEntryID         0
f1                    0
f2               272751
f3               272751
f4               272751
f5               272751
f6               272751
f7               272751
f8               335265
f9               335265
f10              335265
f11              335265
f12              335265
f13              335265
f14              335265
f15              335265
f16              335265
f17              335265
f18              335265
f19              335265
f20              272751
f21              335263
f22              335263
f23              335265
f24              272751
f25              272751
f26              272751
f27              272751
f28              272751
f29                   0
f30                   0
f31                   0
f32                   0
f33                   0
f34              607544
f35              607544
dtype: int64

In [9]:
motion.head()

,user,datetime,motionEntryID,f1,f2,f3,f4,f5,f6,f7,...,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35
8688,31,2016-01-17 21:00:00,11344,0.833333,10.138893,1.808323,4.803497,0.052443,5.080081,14.445088,...,0.003922,0.066667,0.129412,0.013298,399.288773,1798.711227,0.000000,0.000000,NaN,NaN
8689,31,2016-01-17 22:00:00,11345,0.875000,9.991126,0.851300,4.342317,0.516907,8.313152,12.166301,...,0.113725,0.105882,0.054902,0.028916,1000.728618,703.500463,534.149730,1.873498,NaN,NaN
8690,31,2016-01-17 23:00:00,11346,0.916667,9.917067,1.318472,4.497558,-0.432710,5.684687,12.710672,...,0.117647,0.070588,0.003922,0.009672,783.605152,717.445904,867.905747,0.902869,NaN,NaN
8715,34,2016-01-18 16:00:00,11371,0.625000,9.936231,1.996349,2.600708,0.272452,6.361103,14.148081,...,0.172549,0.215686,0.188235,0.039071,1473.622621,868.230452,1702.554510,0.865536,NaN,NaN
8716,34,2016-01-18 17:00:00,11372,0.666667,9.897368,0.469227,3.396609,0.012237,8.796297,11.034094,...,0.231373,0.211765,0.152941,0.424630,151.908247,42.051608,11.842664,12.827202,NaN,NaN
